# GPT2 Language Model Training

In this notebook, we will train our model on sample data to estimate its runnig time.


In [1]:
import os
import wandb
import torch
import random
import numpy as np
import pandas as pd
from rich import print
from pathlib import Path
from tqdm.auto import tqdm
from dotenv import load_dotenv
from collections import defaultdict
from huggingface_hub import login

In [2]:
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset
import transformers
from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
sns.set_theme(context='notebook', style='dark')
tqdm.pandas()

In [5]:
# fix random seed
seed = 42

random.seed(seed)

np.random.seed(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

transformers.set_seed(seed)

In [6]:
dataset_path = Path('../data/sentences.txt')
tokenizer_pah = Path('../models/nano-gpt-tokenizer/')
model_ckpt = 'openai-community/gpt2'
model_name = 'arabic-nano-gpt'
model_path = Path(f'../models/{model_name}')

In [7]:
print(model_path)

../models/arabic-nano-gpt

In [8]:
# wandb configs
PROJECT_NAME = 'Arabic-Nano-GPT'
JOB_TYPE = 'LM-Modeling'
TAGS = ['MOdeling', 'Transformers', 'GPT2', 'Language-Modeling', 'Arabic']
NOTES = 'LM Training on Arabic Data using GPT2 Model Architecture'
RUN_NAME = 'Dummy-data-and-model-LM-Moldeing'
config = defaultdict(dict)

In [9]:
load_dotenv()
wandb.login()
login(token=os.getenv('HF_TOKEN'),
      add_to_git_credential=True, write_permission=True)  # HF

wandb: Currently logged in as: e_hossam96. Use `wandb login --relogin` to force relogin


Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/hossam/.cache/huggingface/token
Login successful


In [10]:
# load the dataset
with open(dataset_path, 'r') as f:
    dataset = f.readlines()[:100_000]

In [11]:
train, valid = train_test_split(dataset, train_size=0.8, random_state=seed)
valid, test = train_test_split(valid, train_size=0.5, random_state=seed)

In [12]:
dataset = DatasetDict()
dataset['train'] = Dataset.from_dict({'text': train})
dataset['valid'] = Dataset.from_dict({'text': valid})
dataset['test'] = Dataset.from_dict({'text': test})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 80000
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 10000
    })
})

In [13]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_pah)
print(tokenizer)

GPT2TokenizerFast(name_or_path='../models/nano-gpt-tokenizer', vocab_size=4096, model_max_length=128, is_fast=True,
padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': 
'<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
        0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, 
special=True),
}

In [14]:
def tokenize(batch):
    outputs = tokenizer(
        batch['text'],
        truncation=True,
        return_overflowing_tokens=True,
    )
    return {'input_ids': outputs['input_ids']}


tokenized_dataset = dataset.map(
    tokenize, batched=True, remove_columns=dataset['train'].column_names
)
print(tokenized_dataset)

Parameter 'function'=<function tokenize at 0x7fca4d026de0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 80056
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 10008
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 10002
    })
})

In [15]:
tokenizer.decode(tokenized_dataset['train'][0]['input_ids'])

'الحضارات القديمة في أمريكا الشمالية\n'

In [16]:
model_config = AutoConfig.from_pretrained(
    model_ckpt, vocab_size=tokenizer.vocab_size,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    n_positions=tokenizer.model_max_length,
    n_ctx=tokenizer.model_max_length,
    n_embd=128, n_head=2, n_layer=2,
)
print(model_config)

GPT2Config {
  "_name_or_path": "openai-community/gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 128,
  "n_embd": 128,
  "n_head": 2,
  "n_inner": null,
  "n_layer": 2,
  "n_positions": 128,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 4096
}

In [17]:
model = AutoModelForCausalLM.from_config(model_config)
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(4096, 128)
    (wpe): Embedding(128, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-1): 2 x GPT2Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=128, out_features=4096, bias=False)
)

In [18]:
print('Size in MBs:', model.get_memory_footprint() / 1_000_000)

Size in MBs: 3.782664

In [19]:
print('Num Params:', model.num_parameters() / 1_000_000, 'M')

Num Params: 0.937472 M

In [20]:
num_epochs = 5
batch_size = 16 * 32
total_steps = len(tokenized_dataset['train']) * num_epochs // batch_size
total_steps

781

In [21]:
training_args = TrainingArguments(
    output_dir=model_path,
    run_name=RUN_NAME,
    report_to='wandb',
    save_strategy='no',
    eval_strategy='steps',
    gradient_accumulation_steps=32,
    overwrite_output_dir=True,
    data_seed=seed, seed=seed,
    learning_rate=1e-3,
    weight_decay=0.001,
    warmup_ratio=0.0,
    eval_steps=500,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
)

In [22]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['valid'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [24]:
run = wandb.init(project=PROJECT_NAME, job_type=JOB_TYPE,
                 name=RUN_NAME, notes=NOTES, tags=TAGS, config=config)

In [25]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=20, training_loss=7.696140289306641, metrics={'train_runtime': 1990.4362, 'train_samples_per_second': 201.102, 'train_steps_per_second': 0.01, 'total_flos': 86382433075200.0, 'train_loss': 7.696140289306641, 'epoch': 4.076433121019108})

In [26]:
trainer.evaluate(tokenized_dataset['test'], metric_key_prefix='test')

{'test_loss': 7.391066551208496,
 'test_runtime': 7.5792,
 'test_samples_per_second': 1319.661,
 'test_steps_per_second': 2.639,
 'epoch': 4.076433121019108}

In [27]:
wandb.finish()

test/loss,▁
test/runtime,▁
test/samples_per_second,▁
test/steps_per_second,▁
train/epoch,▁▁
train/global_step,▁▁
test/loss,7.39107
test/runtime,7.5792
test/samples_per_second,1319.661
test/steps_per_second,2.639
total_flos,86382433075200.0


In [28]:
trainer.save_model(model_path)

In [29]:
_ = trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/3.75M [00:00<?, ?B/s]